## Imports

In [58]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [59]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## Create a new table datamart in the database by joining the tables pageviews and checker using only one query
◦ the table should have the following columns: uid, labname, first_commit_ts, first_view_ts

◦ first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user

◦ first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed

◦ status = ’ready’ should still be a filter

◦ numTrials = 1 should still be a filter

◦ labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’

◦ the table should contain only the users (uids with user_*) and not the admins

◦ first_commit_ts and first_view_ts should be parsed as datetime64[ns]

In [60]:
sel="""
CREATE TABLE datamart AS
    SELECT c.uid, c.labname, c.first_commit_ts, p.first_view_ts FROM
        (SELECT uid, labname, timestamp AS first_commit_ts FROM checker
            WHERE uid LIKE 'user_%'
                AND status LIKE 'ready'
                AND numTrials = 1
                AND labname IN ('laba04', 'laba04s', 'laba05',
                                'laba06', 'laba06s', 'project1')
        ) AS c
    LEFT JOIN
        (SELECT uid, MIN(datetime) as first_view_ts FROM pageviews
            GROUP BY uid
        ) AS p
    ON c.uid = p.uid;"""

In [61]:
cur = conn.cursor()

In [62]:
cur.execute('DROP TABLE IF EXISTS datamart;')

In [63]:
cur.execute(sel)

In [64]:
datamart = pd.read_sql(sql='SELECT * FROM datamart;', 
					   con=conn, 
					   parse_dates=['first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


## Using Pandas methods, create two dataframes: test and control
◦ test should have the users that have the values in first_view_ts

In [65]:
test = datamart.loc[datamart.first_view_ts.notnull()]
test.head()

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


◦ control should have the users that have missing values in first_view_ts

In [66]:
control = datamart.loc[datamart.first_view_ts.isnull()]
control.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
5,user_2,laba04,2020-04-18 13:42:35.482008,NaT
6,user_2,laba04s,2020-04-18 13:51:22.291271,NaT


◦ replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis

In [67]:
control.fillna(test.first_view_ts.mean(), inplace=True)
control.head()

/Users/maksimsavin/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552


In [68]:
control.describe

<bound method NDFrame.describe of          uid   labname            first_commit_ts  \
0     user_4  project1 2020-04-17 05:19:02.744528   
1     user_4    laba04 2020-04-17 11:33:17.366400   
2     user_4   laba04s 2020-04-17 11:48:41.992466   
5     user_2    laba04 2020-04-18 13:42:35.482008   
6     user_2   laba04s 2020-04-18 13:51:22.291271   
..       ...       ...                        ...   
126   user_2   laba06s 2020-05-19 14:45:03.908268   
132   user_6   laba06s 2020-05-20 14:50:07.609937   
134   user_7   laba06s 2020-05-20 23:05:37.742597   
135  user_23    laba06 2020-05-21 08:34:10.517205   
137  user_23   laba06s 2020-05-21 14:29:15.709568   

                    first_view_ts  
0   2020-04-27 00:40:05.761783552  
1   2020-04-27 00:40:05.761783552  
2   2020-04-27 00:40:05.761783552  
5   2020-04-27 00:40:05.761783552  
6   2020-04-27 00:40:05.761783552  
..                            ...  
126 2020-04-27 00:40:05.761783552  
132 2020-04-27 00:40:05.761783552  
134 2

◦ save both tables into the database, you will use them in the next exercises

In [69]:
test.to_sql('test', conn, if_exists='replace')
control.to_sql('control', conn, if_exists='replace')

In [70]:
pd.read_sql('SELECT * FROM test LIMIT 5;', conn)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


In [71]:
pd.read_sql('SELECT * FROM control LIMIT 5;', conn)

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783
1,1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783
2,2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783
3,5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783
4,6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783


## Close the connection

In [72]:
conn.close()